# Summarizing failure reasons
This notebook demonstrates how one can dive into summarizing error messages and failure reasones. The `_result.jsonl` files contain a column `result`, which contains a string, that in case of test failure is "failed: " and in other cases contains additionally the error message that was observed. These failures and errors can be summarized for each model as shown here.

In [1]:
import pandas as pd
import os

In [2]:
directory = "../data/"
# if you want to investigate a single model only, add its name here:
search_term = ""

# Enter the terms to search for here
common_errors = ['has no attribute', 'invalid syntax', 'is not defined', "Can't convert object", 'cannot import', 'out of range', 'unexpected keyword argument']

First we collect all results and the corresponding models from the jsonl files.

In [3]:
collection = []
for filename in os.listdir(directory):
    if search_term in filename and filename.endswith("_results.jsonl"):
        df = pd.read_json(directory + filename, lines=True)
        df['model'] = filename.replace("samples_","").replace("_results","").replace(".jsonl","")
        collection.append(df)

if len(collection) == 1:
    df = collection[0]
else:
    df = pd.concat(collection)

We then focus on the tests which failed.

In [4]:
df = df[df['passed'] == False]
df

,task_id,completion,full_response,result,passed,model
0,../test_cases/apply_otsu_threshold_and_count_p...,\nimport cv2\nimport numpy as np\n\ndef apply_...,Here's the completed code with a step-by-step ...,failed: OpenCV(4.9.0) d:\a\opencv-python\openc...,False,claude-3-opus-20240229
2,../test_cases/binary_skeleton.ipynb,\nimport numpy as np\nimport cv2\n\ndef binary...,Here's the completed code with a step-by-step ...,failed: module 'cv2' has no attribute 'ximgproc',False,claude-3-opus-20240229
4,../test_cases/combine_columns_of_tables.ipynb,\n# Plan:\n# 1. Check if the index column is p...,"Here's the completed code with comments, impor...",failed: Can only compare identically-labeled (...,False,claude-3-opus-20240229
7,../test_cases/count_number_of_touching_neighbo...,\nimport numpy as np\nfrom scipy.ndimage impor...,"Here's the completed code with a plan, necessa...",failed:,False,claude-3-opus-20240229
8,../test_cases/count_objects_over_time.ipynb,\ndef count_objects_over_time(binary_image_lis...,```python\ndef count_objects_over_time(binary_...,failed:,False,claude-3-opus-20240229
...,...,...,...,...,...,...
558,../test_cases/subtract_background_tophat.ipynb,\n# Plan\n# 1. Import the necessary libraries:...,```python\n# Plan\n# 1. Import the necessary l...,failed: white_tophat() got an unexpected keywo...,False,gpt-4-turbo-2024-04-09
562,../test_cases/transpose_image_axes.ipynb,"\ndef transpose_image_axes(image):\n """"""\n ...",```python\ndef transpose_image_axes(image):\n ...,failed: axes don't match array,False,gpt-4-turbo-2024-04-09
564,../test_cases/workflow_batch_process_folder_co...,\n# Plan:\n# 1. Use os to list all files in pr...,```python\n# Plan:\n# 1. Use os to list all fi...,failed:,False,gpt-4-turbo-2024-04-09
565,../test_cases/workflow_batch_process_folder_me...,\n# Plan:\n# 1. Import the necessary libraries...,```python\n# Plan:\n# 1. Import the necessary ...,failed: The dimensions of the image and the la...,False,gpt-4-turbo-2024-04-09


In [5]:
# Just some examples to get an idea
df.head(10)['result'].tolist()

["failed: OpenCV(4.9.0) d:\\a\\opencv-python\\opencv-python\\opencv\\modules\\imgproc\\src\\color.simd_helpers.hpp:92: error: (-2:Unspecified error) in function '__cdecl cv::impl::`anonymous-namespace'::CvtHelper<struct cv::impl::`anonymous namespace'::Set<3,4,-1>,struct cv::impl::A0x59191d0d::Set<1,-1,-1>,struct cv::impl::A0x59191d0d::Set<0,2,5>,4>::CvtHelper(const class cv::_InputArray &,const class cv::_OutputArray &,int)'\n> Invalid number of channels in input image:\n>     'VScn::contains(scn)'\n> where\n>     'scn' is 1\n",
 "failed: module 'cv2' has no attribute 'ximgproc'",
 'failed: Can only compare identically-labeled (both index and columns) DataFrame objects',
 'failed: ',
 'failed: ',
 'failed: ',
 "failed: OpenCV(4.9.0) d:\\a\\opencv-python\\opencv-python\\opencv\\modules\\imgproc\\src\\color.simd_helpers.hpp:92: error: (-2:Unspecified error) in function '__cdecl cv::impl::`anonymous-namespace'::CvtHelper<struct cv::impl::`anonymous namespace'::Set<3,4,-1>,struct cv::impl

## Searching for common terms
First, we search the error messages for common errors as specified above.

In [6]:
# Define the function to count errors
def count_errors(group, error_list):
    counts = {error: group['result'].str.contains(error, regex=False).sum() for error in error_list}
    return pd.Series(counts)

# Apply the function to each model group
error_counts = df.groupby('model').apply(count_errors, error_list=common_errors)

# Transpose the result for the desired format: models as columns, errors as rows
error_counts = error_counts.T

C:\Users\haase\AppData\Local\Temp\ipykernel_17672\939699814.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  error_counts = df.groupby('model').apply(count_errors, error_list=common_errors)


In [7]:
error_counts

model,claude-3-opus-20240229,codellama,gemini-pro,gpt-3.5-turbo-1106,gpt-4-1106-preview,gpt-4-turbo-2024-04-09
has no attribute,45,59,43,37,48,33
invalid syntax,0,58,0,4,1,0
is not defined,5,32,203,11,8,4
Can't convert object,1,13,3,9,3,1
cannot import,5,17,6,2,2,3
out of range,3,4,0,0,0,0
unexpected keyword argument,5,4,1,7,8,15


In [8]:
error_counts.columns = [f"\\rot{{{col}}}" for col in error_counts.columns]
error_counts

,\rot{claude-3-opus-20240229},\rot{codellama},\rot{gemini-pro},\rot{gpt-3.5-turbo-1106},\rot{gpt-4-1106-preview},\rot{gpt-4-turbo-2024-04-09}
has no attribute,45,59,43,37,48,33
invalid syntax,0,58,0,4,1,0
is not defined,5,32,203,11,8,4
Can't convert object,1,13,3,9,3,1
cannot import,5,17,6,2,2,3
out of range,3,4,0,0,0,0
unexpected keyword argument,5,4,1,7,8,15


Next we save the dataframe as tex to disk, so that we can include it in the manuscript.

In [9]:
# Convert DataFrame to TeX format string
tex_str = error_counts.to_latex()

# Define filename for the TeX file
tex_filename = '../docs/paper/error_counts_table.tex'

# Write the TeX format string to a file
with open(tex_filename, 'w') as tex_file:
    tex_file.write(tex_str)

## Most popular failure reasons
Furthermore, we search for the three most observed reasons for failure. These might be either error messages, or in case the result is only `failed: ` this indicated that the tests were not passed, presumably because the tested function did not return the right result.

In [10]:
# Step 1: Group the DataFrame by 'model' and get the value counts of 'result'
model_result_count = df.groupby('model')['result'].value_counts()

# Step 2: Create an empty DataFrame to store the results
model_top_results = []

# Step 3: Loop through each group to get the three most common results per model
for model, counts in model_result_count.groupby(level=0):
    # Get the top three results (note: nlargest returns the results)
    top_three = counts.nlargest(3)
    # Prepare data to append to the DataFrame
    data = {
        'Model': model,
        'Top1 Result': top_three.index.get_level_values(1)[0],
        'Top1 Count': top_three.iloc[0],
        'Top2 Result': top_three.index.get_level_values(1)[1] if len(top_three) > 1 else None,
        'Top2 Count': top_three.iloc[1] if len(top_three) > 1 else None,
        'Top3 Result': top_three.index.get_level_values(1)[2] if len(top_three) > 2 else None,
        'Top3 Count': top_three.iloc[2] if len(top_three) > 2 else None
    }
    # Append data
    model_top_results.append(data)

# Display the resulting DataFrame
most_common_errors = pd.DataFrame(model_top_results)
most_common_errors

,Model,Top1 Result,Top1 Count,Top2 Result,Top2 Count,Top3 Result,Top3 Count
0,claude-3-opus-20240229,failed:,154,failed: 'list' object has no attribute 'shape',23,failed: OpenCV(4.9.0) d:\a\opencv-python\openc...,11
1,codellama,failed:,97,failed: OpenCV(4.9.0) d:\a\opencv-python\openc...,26,"failed: invalid syntax (<string>, line 4)",22
2,gemini-pro,failed: name 'np' is not defined,152,failed:,91,failed: 'list' object has no attribute 'shape',17
3,gpt-3.5-turbo-1106,failed:,143,failed: OpenCV(4.9.0) D:\a\opencv-python\openc...,19,failed: OpenCV(4.9.0) d:\a\opencv-python\openc...,16
4,gpt-4-1106-preview,failed:,141,failed: 'list' object has no attribute 'shape',16,failed: 'numpy.ndarray' object has no attribut...,12
5,gpt-4-turbo-2024-04-09,failed:,151,failed: OpenCV(4.9.0) d:\a\opencv-python\openc...,15,failed: 'list' object has no attribute 'shape',14
